<a href="https://colab.research.google.com/github/rickygrosvenor-pramanick/ray-gen/blob/main/ray_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install diffusers transformers accelerate bitsandbytes safetensors datasets peft torch torchvision safetensors huggingface_hub

In [10]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet").to(device)
lora_cfg = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,  # generic fallback
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["to_q","to_k","to_v"]
)
unet_lora = get_peft_model(unet, lora_cfg).to(device)